In [1]:
%%capture
!pip install duckduckgo-search
!pip install fastai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [15]:
%%capture
import certifi
import urllib.request
import ssl
from duckduckgo_search import DDGS
from fastai.vision.all import *
from fastai.vision.widgets import *

DATA_DIR = Path("data")
MODEL_PATH = Path("exercise_classifier.pkl")

def download_image_with_headers(url: str, dest: Path):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    req = urllib.request.Request(url, headers=headers)
    ctx = ssl.create_default_context(cafile=certifi.where())

    with urllib.request.urlopen(req, context=ctx) as response:
        dest.write_bytes(response.read())

def search_images_ddg(keywords, max_images = 30):
    return L(DDGS().images(keywords, max_results=max_images)).itemgot("image")

def download(label: str, n: int = 30):
    dest = DATA_DIR/label
    dest.mkdir(parents=True, exist_ok=True)
    urls = search_images_ddg(f"{label} photo", max_images = n)
    for i, url in enumerate(urls):
        try:
            img_path = dest/f"{label}_{i}.jpg"
            download_image_with_headers(url, img_path)
        except Exception as e:
            print(f"Failed: {url} - {e}")
    print(f"Downloaded {len(dest.ls())} images to {dest}")

In [22]:
%%capture
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [23]:
%%capture
learn_inf = load_learner(path/'export.pkl')

/Users/vadstart/Documents/Projects/ML/FastAI/02-fastai-jupyter/.venv/lib/python3.12/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [63]:
import io
from ipywidgets import FileUpload, Button, Output, Label, VBox, HBox
from IPython.display import display

# Upload widget
btn_upload = FileUpload(accept='image/*', multiple=False)
out_pl = Output()
lbl_pred = Label(value='Prediction:')

# Button click handler
def on_click_classify(btn):
    if btn_upload.value:
        print("Classify button clicked!")
        uploaded_file = btn_upload.value[0]

        img_bytes = io.BytesIO(bytes(uploaded_file['content']))
        img = PILImage.create(img_bytes)

        out_pl.clear_output()
        with out_pl:
            display(img.to_thumb(128,128))

        # Replace with real model prediction logic
        pred, pred_idx, probs = learn_inf.predict(img)
        lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    else:
        print("No file uploaded.")

# Classify button
btn_run = Button(description='Classify')
btn_run.on_click(on_click_classify)

# Show all widgets
display(VBox([btn_upload, btn_run, out_pl, lbl_pred]))